# Analiza danych dotyczących pandemi covid-19

W poniższym raporcie przestawiono proces opracowania i wykonania analizy opartej o dostępne dane dotycznące pandemi wirusa covid-19.

## Zależności 

Poniżej zaimportowano wymagane biblioteki języka Python wymagane do uruchomienia analizy.

In [402]:
import pandas as pd
import numpy as np
import requests
import datetime
import matplotlib.pyplot as plt
import squarify
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import cufflinks as cf
import palettable
import plotly.express as px
import plotly.graph_objects as go
import pycountry
import gettext

Następnie wczytano moduły, które będą konieczne do wykonania tłumaczeń oraz poprawnego wyświetlania wykresów.

In [403]:
## Tłumaczenia
poland = gettext.translation('iso3166', pycountry.LOCALES_DIR, languages=['pl'])
poland.install()

# Wykresy
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

## Dane źródłowe

Jako dane źródłowe w poniższej analizie posłużą dane zebrane przez zespół badawczy CSSE (Center for Systems Science and Engineering) z Uniwersystetu Johna Hopkinsa. Dane udostępnione są na licencji Creative Commons Attribution 4.0 International (CC BY 4.0) i są zagregowanymi danymi pochodzącymi z różnych źródeł (więcej informacji o pochodzeniu danych dostępne na stronie [repozytorium](https://github.com/CSSEGISandData/COVID-19) i są aktualizowane raz dziennie.

Dane zostały pobrane w najnowszej dostępnej wersji. Na dzień analizy najnowszy uwzględniony dzień to 7 stycznia 2021.

Dla ułatwienia procesu pobierania danych (kilka arkuszy) przygotowano prostą funkcję, która pozwala na pobranie i zapisanie pliku z danymi.

In [404]:
def downloadData(url):
    r = requests.get(url, allow_redirects=True)
    filename = url.rsplit('/', 1)[1]
    open(filename, 'wb').write(r.content)

Następnie wywołano funkcję dla wymaganych arkuszy z danymi.

In [405]:
downloadData('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')    
downloadData('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
downloadData('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
downloadData('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/UID_ISO_FIPS_LookUp_Table.csv')

I wczytano arkusze do stosownych zmiennych.

In [406]:
df_confirmed = pd.read_csv('time_series_covid19_confirmed_global.csv')
df_deaths = pd.read_csv('time_series_covid19_deaths_global.csv')
df_recovered = pd.read_csv('time_series_covid19_recovered_global.csv')
df_countries = pd.read_csv('UID_ISO_FIPS_LookUp_Table.csv')

## Przegląd danych 

Główne dane na których będzie sie opierała analiza składają się z trzech osobnych tabel zawierających sumy przypadków (potwierdzonych zachorowań, przypadków śmiertelnych, ozdrowień) z podziałem na kraje (dla niektórych krajów również regiony) dla każdego z dnia pandemii. 

Przypadki zdefiniowane sa jako:
- potwierdzone: liczba zawiera przypadki potwierdzone i prawdopodobne (gdzie zgłoszono)
- śmiertelne: liczba zawiera przypadki potwierdzone i prawdopodobne (gdzie zgłoszono).
- ozdrowienia: liczba szacowana na podstawie danych z lokalnych mediów, i raportów jeśli dostępne - może być zaniżona w stosunku do realnych danych

W każdej z tabel występuje szereg kolumn.

In [407]:
df_confirmed.columns

Index(['Province/State', 'Country/Region', 'Lat', 'Long', '1/22/20', '1/23/20',
       '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       ...
       '12/30/20', '12/31/20', '1/1/21', '1/2/21', '1/3/21', '1/4/21',
       '1/5/21', '1/6/21', '1/7/21', '1/8/21'],
      dtype='object', length=357)

Pierwsze cztery z kolumn 'Province/State', 'Country/Region', 'Lat', 'Long' zawierają dane dotyczące danej lokalizacji, pozostałe kolumny zawierają liczbę całkowitą zaraportowanych przypadków - nazwa kolumn w strukturze miesiąc / dzień / rok.

In [408]:
df_confirmed.dtypes

Province/State     object
Country/Region     object
Lat               float64
Long              float64
1/22/20             int64
                   ...   
1/4/21              int64
1/5/21              int64
1/6/21              int64
1/7/21              int64
1/8/21              int64
Length: 357, dtype: object

Przykładowa zawartość jednej z kolumn z danymi - kolumna dla dnia 5 stycznia 2021 - przypadki potwierdzone.

In [409]:
df_confirmed['1/5/21'].head(10)

0      53105
1      60283
2     100873
3       8308
4      17756
5        160
6    1662730
7     160544
8        118
9       4978
Name: 1/5/21, dtype: int64

In [410]:
df_confirmed['1/5/21'].describe()

count    2.720000e+02
mean     3.176814e+05
std      1.547226e+06
min      0.000000e+00
25%      7.285000e+02
50%      9.224500e+03
75%      1.287210e+05
max      2.104960e+07
Name: 1/5/21, dtype: float64

Dodatkowo zaimportowana tabela zawierająca spis krajów wraz z podziałem administracyjnym oraz populacją dla danego regionu. Pozostałe dane są nadmiarowe i nie są używane w poniższym raporcie.

In [411]:
df_countries.columns

Index(['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Province_State',
       'Country_Region', 'Lat', 'Long_', 'Combined_Key', 'Population'],
      dtype='object')

In [412]:
df_countries

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Population
0,4,AF,AFG,4.0,NaN,NaN,NaN,Afghanistan,33.939110,67.709953,Afghanistan,38928341.0
1,8,AL,ALB,8.0,NaN,NaN,NaN,Albania,41.153300,20.168300,Albania,2877800.0
2,12,DZ,DZA,12.0,NaN,NaN,NaN,Algeria,28.033900,1.659600,Algeria,43851043.0
3,20,AD,AND,20.0,NaN,NaN,NaN,Andorra,42.506300,1.521800,Andorra,77265.0
4,24,AO,AGO,24.0,NaN,NaN,NaN,Angola,-11.202700,17.873900,Angola,32866268.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4166,84056037,US,USA,840.0,56037.0,Sweetwater,Wyoming,US,41.659439,-108.882788,"Sweetwater, Wyoming, US",42343.0
4167,84056039,US,USA,840.0,56039.0,Teton,Wyoming,US,43.935225,-110.589080,"Teton, Wyoming, US",23464.0
4168,84056041,US,USA,840.0,56041.0,Uinta,Wyoming,US,41.287818,-110.547578,"Uinta, Wyoming, US",20226.0
4169,84056043,US,USA,840.0,56043.0,Washakie,Wyoming,US,43.904516,-107.680187,"Washakie, Wyoming, US",7805.0


## Wstępna obróbka danych

W przypadku tabel z danymi zdecydowano się o usunięciu nadmiarowych kolumn zawierajacych współrzędne geograficzne i podział administracyjny - pozostawiając podział na kraj/region oraz właściwe dane.

In [413]:
def removeColumns(df, columns):
    try:
        df.drop(columns, inplace=True, axis=1)
    except:
        pass
    
removeColumns(df_confirmed, ['Lat', 'Long', 'Province/State'])
removeColumns(df_deaths, ['Lat', 'Long', 'Province/State'])
removeColumns(df_recovered, ['Lat', 'Long', 'Province/State'])
removeColumns(df_countries, ['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Province_State', 'Lat', 'Long_', 'Combined_Key'])

Dane źródłowe zawierają nazwy krajów w języku angielskim, stąd na potrzeby raportu zdecydowano o wykonaniu tłumaczenia nazw na język polski na potrzeby niniejszego raportu. 

Równocześnie zdecydowano o przeformatowaniu nazw kolumn zawierających datę w formacie miesiąc/dzień/rok do formatu rok/miesiac/dzień dla ułatwienia odczytywania daty.

In [414]:
df_countries['Country_Region'] = df_countries.apply(lambda row: _(row['Country_Region']), axis=1)

In [415]:
def formatDateColumnNames(df):
    columns = df.columns
    df['Country/Region'] = df.apply(lambda row: _(row['Country/Region']), axis=1)
    for column in columns:
        try:
            df.rename(columns={column: datetime.datetime.strptime(column, '%m/%d/%y').strftime('%Y/%m/%d')}, inplace=True)
        except:
            pass

formatDateColumnNames(df_confirmed)
formatDateColumnNames(df_deaths)
formatDateColumnNames(df_recovered)

Po wykonaniu operacji struktura tabel z danymi:

In [416]:
df_confirmed.columns

Index(['Country/Region', '2020/01/22', '2020/01/23', '2020/01/24',
       '2020/01/25', '2020/01/26', '2020/01/27', '2020/01/28', '2020/01/29',
       '2020/01/30',
       ...
       '2020/12/30', '2020/12/31', '2021/01/01', '2021/01/02', '2021/01/03',
       '2021/01/04', '2021/01/05', '2021/01/06', '2021/01/07', '2021/01/08'],
      dtype='object', length=354)

Oraz struktura tabeli z populacją i krajami:

In [417]:
df_countries.columns

Index(['Country_Region', 'Population'], dtype='object')

Przetłumaczone nazwy krajów w tabeli:

In [418]:
df_confirmed['Country/Region'].unique()

array(['Afganistan', 'Albania', 'Algieria', 'Andora', 'Angola',
       'Antigua i Barbuda', 'Argentyna', 'Armenia', 'Australia',
       'Austria', 'Azerbejdżan', 'Bahamy', 'Bahrajn', 'Bangladesz',
       'Barbados', 'Białoruś', 'Belgia', 'Belize', 'Benin', 'Bhutan',
       'Boliwia', 'Bośnia i Hercegowina', 'Botswana', 'Brazylia',
       'Brunei', 'Bułgaria', 'Burkina Faso', 'Burma', 'Burundi',
       'Cabo Verde', 'Kambodża', 'Kamerun', 'Kanada',
       'Republika Środkowoafrykańska', 'Czad', 'Chile', 'Chiny',
       'Kolumbia', 'Komory', 'Congo (Brazzaville)', 'Congo (Kinshasa)',
       'Kostaryka', "Cote d'Ivoire", 'Chorwacja', 'Kuba', 'Cypr',
       'Czechia', 'Dania', 'Diamond Princess', 'Dżibuti', 'Dominika',
       'Dominikana', 'Ekwador', 'Egipt', 'Salwador', 'Gwinea Równikowa',
       'Erytrea', 'Estonia', 'Eswatini', 'Etiopia', 'Fidżi', 'Finlandia',
       'Francja', 'Gabon', 'Gambia', 'Gruzja', 'Niemcy', 'Ghana',
       'Grecja', 'Grenada', 'Gwatemala', 'Gwinea', 'Gwinea Bis

Następnie przygotowano pomocniczą listę zawierajacą daty raportów.

In [419]:
# Przygotowanie indeksu dat
l_dates = df_confirmed.columns.tolist()
l_dates.pop(0)
l_dates

['2020/01/22',
 '2020/01/23',
 '2020/01/24',
 '2020/01/25',
 '2020/01/26',
 '2020/01/27',
 '2020/01/28',
 '2020/01/29',
 '2020/01/30',
 '2020/01/31',
 '2020/02/01',
 '2020/02/02',
 '2020/02/03',
 '2020/02/04',
 '2020/02/05',
 '2020/02/06',
 '2020/02/07',
 '2020/02/08',
 '2020/02/09',
 '2020/02/10',
 '2020/02/11',
 '2020/02/12',
 '2020/02/13',
 '2020/02/14',
 '2020/02/15',
 '2020/02/16',
 '2020/02/17',
 '2020/02/18',
 '2020/02/19',
 '2020/02/20',
 '2020/02/21',
 '2020/02/22',
 '2020/02/23',
 '2020/02/24',
 '2020/02/25',
 '2020/02/26',
 '2020/02/27',
 '2020/02/28',
 '2020/02/29',
 '2020/03/01',
 '2020/03/02',
 '2020/03/03',
 '2020/03/04',
 '2020/03/05',
 '2020/03/06',
 '2020/03/07',
 '2020/03/08',
 '2020/03/09',
 '2020/03/10',
 '2020/03/11',
 '2020/03/12',
 '2020/03/13',
 '2020/03/14',
 '2020/03/15',
 '2020/03/16',
 '2020/03/17',
 '2020/03/18',
 '2020/03/19',
 '2020/03/20',
 '2020/03/21',
 '2020/03/22',
 '2020/03/23',
 '2020/03/24',
 '2020/03/25',
 '2020/03/26',
 '2020/03/27',
 '2020/03/

## Analiza danych

Ze względu na dużą liczbę dostępnych danych zdecydowano sie na przygotowanie interaktywnych wykresów pozwalających ograniczyć liczbę wyświetlanych danych. W zależności od wykresu będzie to kraj, rodzaj przypadków lub przedział występowania.

Analiza została przeprowadzona na dzień 7 stycznia 2021 stąd ze względu na ciągłą aktualizację danych źródłowych komentarze dotyczące poszczególnych zaobserwowanych zależności mogą nie w przyszłości nie odpowiadać wykresom oraz omawianym danym.


### Całkowita liczba przypadków covid-19 w wybranym kraju w danym dniu.

Poniższy wykres pozwala na wybranie kraju oraz rodzaju przypadku (potwierdzone, śmiertelne, wyzdrowienia) i obejrzenie wykresu przedstawiającego całkowitą jego liczbę w poszczególnych dniach. 

W domyślnej konfiguracji wykres został ustawiony dla kraju Polska oraz przypadków potwierdzonych. Poniżej wykresu przedstawiono wnioski z analizy wykresu.

In [426]:
@interact
def show_cases_by_country_type(
    country=widgets.Dropdown(
        options=sorted(df_countries['Country_Region'].unique()), 
        value=_("Poland"),
        description='Kraj:',
    ),
    typ=widgets.Dropdown(
        options=['potwierdzone', 'śmiertelne', 'wyzdrowienia'], 
        value='potwierdzone',
        description="Rodzaj przypadku:"
    )):
    
    # Choose correct dataframe and corresponding color
    if typ == 'potwierdzone':  
        df = df_confirmed
        color = ['#636efa']
    elif typ == 'śmiertelne':
        df = df_deaths
        color = ['#ef553b']
    else:
        df = df_recovered
        color = ['#00cc96']
    
    # Group data by countries (if country data is provided with regions)
    return pd.pivot_table(df.loc[df['Country/Region'] == country], 
                          columns=['Country/Region'], 
                          aggfunc=sum, 
                          fill_value=0
                        ).iplot(
                            kind="bar", 
                            theme="white", 
                            color=color, 
                            title='Suma '+typ+" przypadki covid-19 w " + _(country))

interactive(children=(Dropdown(description='Kraj:', index=135, options=('Afganistan', 'Albania', 'Algieria', '…

Z wykresu sumy potwierdzonych zachorowań dla Polski możemy zauważyć tendencję wzrostową dla sumy zachorowań rozpoczynającą się w okolicach początku października. Podobnie sytuacja wygląda dla przypadków śmiertelnych w wybranym czasie, jednakże zauważyć można przesunięcie wzorstu o parę tygodni.

### Podsumowanie liczby zachorowań dla wybranych krajów

W poniższej analizie przedstawiono wizualnie i liczbowo porównanie sumy zachorowań na covid-19 dla wybranej daty, dla krajów o największej sumie liczby zachorowań.

Interaktywny wykres pozwala na wybór daty, rodzaju przypadku oraz liczby krajów dla którego jest generowana wizualizacja przedstawiająca powierzchniowo ilość dla wybranych przypadku w konteście całego świata.

In [421]:
@interact
def show_squared_top_N_type_country(
    data=widgets.DatePicker(
        disabled=False,
        description='Dzień:',
        value=datetime.date(2021, 1, 7),
    ),
    typ=widgets.Dropdown(
        options=['potwierdzone', 'śmiertelne', 'wyzdrowienia'], 
        value='potwierdzone',
        description='Rodzaj przypadku:'
    ),
    liczba_krajow=widgets.IntSlider(
        value=10,
        min=5,
        max=20,
        step=1,
        description='Liczba krajów:',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='d'
    )):
    
    # Convert data from datePicker to correct format
    data = data.strftime('%Y/%m/%d')

    # Choose correct dataframe
    if typ == 'potwierdzone':  
        df = df_confirmed
    elif typ == 'śmiertelne':
        df = df_deaths
    else:
        df = df_recovered
    
    # Validate date
    if (data not in df.columns):
        return "Data spoza zakresu: "+df.columns[2] + " - " + df.columns[-1]
    
    # Preprocess dataframe (0 values cannot be displayed on squarify plot)
    df = df.groupby(by=["Country/Region"]).sum().sort_values(by=[data], ascending=False)
    df = df.loc[df[data] > 0]
    
    # Rename indexes to have numer of cases below country name
    df = df.rename(index=lambda s: s + '\n' + str(df.loc[s][data]))

    # Append total of not included by name
    all_data = df[data].head(liczba_krajow)
    sum_other = df[data].sum()-df[data].head(liczba_krajow).sum()
    series = pd.Series([sum_other], index=['Pozostałe kraje\n'+str(sum_other)])
    all_data = all_data.append(series)
    
    # Plot the chart
    fig1 = plt.figure()
    ax = fig1.add_subplot()
    fig1.set_size_inches(10, 7)
    squarify.plot(
        sizes=all_data, 
        label=all_data.index, 
        alpha=.8, 
        color=palettable.colorbrewer.qualitative.Paired_10.mpl_colors)
    plt.axis('off')
    plt.title("Wizualizacja liczby potwierdzonych zachorowań\ndla "+str(liczba_krajow)+" krajów z największą liczbą zachorowań \nw dniu "+data,fontsize=23,fontweight="bold")
    plt.show()
        

interactive(children=(DatePicker(value=datetime.date(2021, 1, 7), description='Dzień:'), Dropdown(description=…

Z danych możemy wywnioskować, że najwięcej potwierdzonych zachorowań do dnia 7 stycznia 2021 stwierdzono w USA, liczba ta wyniosiła 21.5miliona, kolejnym w kolejności krajem o najwyższej liczbie potwierdzonych zachorowań są Indie z liczbą ok 10 milionów zachorowań. W dziesiątce krajów najbardziej dotkniętych epidemią nie znajdują się Chiny, gdzie stwierdzono pierwszy przypadek wystąpienia covid-19.

### Łącznie przypadki dla kraju

Poniższy wykres przedstawia skumulowaną liczbę sumy przypadków zachorowań, śmiertelnych i ozdrowień do danego dnia. Wykres pozwala na wybór kraju, dla którego zostanie przedstawiona wizualizacja na wykresie.

In [422]:
@interact
def show_cumulative_by_c(
    country=widgets.Dropdown(
        options=sorted(df_countries['Country_Region'].unique()), 
        value=_("Chiny"),
        description='Kraj:'
    )):
    
    # Group data by countries (if country data is provided with regions)
    data_confirmed_pivot = pd.pivot_table(df_confirmed.loc[df_confirmed['Country/Region'] == country], 
                          columns=['Country/Region'], 
                          aggfunc=sum, 
                          fill_value=0).rename(columns={country: 'potwierdzone'})
    data_confirmed_pivot.reset_index(inplace=True)
    data_deaths_pivot =  pd.pivot_table(df_deaths.loc[df_deaths['Country/Region'] == country], 
                          columns=['Country/Region'], 
                          aggfunc=sum, 
                          fill_value=0).rename(columns={country: 'śmiertelne'})
    data_deaths_pivot.reset_index(inplace=True)
    data_recovered_pivot =  pd.pivot_table(df_recovered.loc[df_recovered['Country/Region'] == country], 
                          columns=['Country/Region'], 
                          aggfunc=sum, 
                          fill_value=0).rename(columns={country: 'wyzdrowienia'})
    data_recovered_pivot.reset_index(inplace=True)
    
    # Merge to single dataframe
    all_data = pd.merge(data_confirmed_pivot, data_deaths_pivot, 'left', on = ["index"] )
    all_data = pd.merge(all_data, data_recovered_pivot, 'left', on = ["index"] )
    all_data.reset_index(inplace=True)
    
    # count active cases
    all_data['aktywne'] = 0
    for i in range(len(all_data)):
        all_data.loc[i,'aktywne'] = all_data.loc[i,'potwierdzone'] - all_data.loc[i,'śmiertelne'] - all_data.loc[i,'wyzdrowienia']
    
    fig = px.bar(
        all_data, 
        x="index", 
        y=["aktywne", "śmiertelne", "wyzdrowienia"], 
        title="Łącznie przypadki covid-19 w "+country, 
        labels={
                     "index": "Dzień",
                     "value": "Całkowita liczba przypadków do dnia",
                     "variable": "Rodzaj przypadku"
                 },
        color_discrete_sequence=['#636efa', '#ef553b', '#00cc96'])
    fig.show()

interactive(children=(Dropdown(description='Kraj:', index=32, options=('Afganistan', 'Albania', 'Algieria', 'A…

Dla wybranego kraju - Chiny, gdzie wykryto pierwsze przypadki występowania wirusa covid-19 -na podstawie analizy wygenerowanego wykresu możemy zauważyć, że od okolic początka marca liczba zachorowań systematycznie spada i utrzymuje się na niskim poziomie. A ogólna liczba zachorowań w momencie analizy jest znacząco mniejsza niż dla krajów z [top10](#Łącznie-przypadki-dla-kraju)

### Dzienny przyrost przypadków covid-19 dla kraju

Wykres przedstawia dzienny przyrost liczby przypadków dla wybranego kraju. Liczba dziennego przyrostu obliczona jest z na podstawie danych o całkowitej liczbie przypadków.

In [423]:
@interact
def show_daily_type_country(
    country=widgets.Dropdown(
        options=sorted(df_countries['Country_Region'].unique()), 
        value=_("Poland"),
        description='Kraj:'
    ),
    typ=widgets.Dropdown(
        options=['potwierdzone', 'śmiertelne', 'wyzdrowienia'], 
        value='potwierdzone',
        description='Rodzaj przypadku:'
    )):
    
    # Choose correct dataframe and color
    if typ == 'potwierdzone':  
        df = df_confirmed
        color = ['#636efa']
    elif typ == 'śmiertelne':
        df = df_deaths
        color = ['#ef553b']
    else:
        df = df_recovered
        color = ['#00cc96']

    # Process dataframe
    df = pd.pivot_table(df.loc[df['Country/Region'] == country], 
                          columns=['Country/Region'], 
                          aggfunc=sum, 
                          fill_value=0
                       ).rename(columns={country: 'total'})
    df.reset_index(inplace=True)
    
    last = None
    df[typ] = 0
    for i in range(len(df)):
        if (i == 0):
            continue
        df.loc[i,typ] = df.loc[i,'total'] - df.loc[i-1,'total']
    
    fig = px.line(
        df, 
        x="index", 
        y=[typ], 
        title="Przypadki "+typ+" (dzienne) covid-19 w "+country, 
        labels={
             "index": "Dzień",
             "value": "Liczba przypadków zaraportowanych w dniu",
             "variable": "Rodzaj przypadku"
        },
        color_discrete_sequence=color)
    fig.show()

interactive(children=(Dropdown(description='Kraj:', index=135, options=('Afganistan', 'Albania', 'Algieria', '…

Z wykresu potwierdzonych nowych przypadków dla każdego dnia dla Polski, możemy zauważyć duży wzrost rozpoczynający się w okolicach połowy września. Największa liczba raportowanych dziennych zachorowań przypada na listopad, gdzie w szytowym momencie zaraportowane zostało nawet 32 tysiące nowych zachorowań w ciągu dnia. Od początku grudnia możemy zaobserwować duże wachania w okolicach niedzieli/poniedziałku - wynikające prawdopodobnie z metody raportowania danych.

Po przełączeniu wykresu na kraj Włochy możemy zaobserwować jak wygląda profil zachorowań dla tego kraju. W porównaniu do Polski - widać dwa wzrosty, pierwszy w okolicach marca 2020, a kolejny w okolicach października. Drugi z wzrostów zbiega się z wzrostami widocznymi na wykresie dla Polski.

### Zachorowania a populacja kraju

Wykres przedstawia liczbę potwierdzonych przypadków zachorowań, śmierci i wyzdrowień na tle pozostałej populacji danego kraju.

In [424]:
@interact
def show_top_10_type_country(
    data=widgets.DatePicker(
        disabled=False,
        description='Dzień:',
        value=datetime.date(2021, 1, 7),
    ),
     country=widgets.Dropdown(
        options=sorted(df_countries['Country_Region'].unique()), 
        value=_("Poland"),
        description='Kraj:'
    )):
    
    # Convert data from datePicker to correct format
    data = data.strftime('%Y/%m/%d')
    
    # Validate date
    if (data not in df_confirmed.columns):
        return "Data spoza zakresu: "+df_confirmed.columns[2] + " - " + df_confirmed.columns[-1]
    
    # Group data by countries (if country data is provided with regions)
    data_confirmed_pivot = pd.pivot_table(df_confirmed.loc[df_confirmed['Country/Region'] == country], 
                          columns=['Country/Region'], 
                          aggfunc=sum, 
                          fill_value=0).rename(columns={country: 'potwierdzone'})
    data_confirmed_pivot.reset_index(inplace=True)
    data_deaths_pivot =  pd.pivot_table(df_deaths.loc[df_deaths['Country/Region'] == country], 
                          columns=['Country/Region'], 
                          aggfunc=sum, 
                          fill_value=0).rename(columns={country: 'śmiertelne'})
    data_deaths_pivot.reset_index(inplace=True)
    data_recovered_pivot =  pd.pivot_table(df_recovered.loc[df_recovered['Country/Region'] == country], 
                          columns=['Country/Region'], 
                          aggfunc=sum, 
                          fill_value=0).rename(columns={country: 'wyzdrowienia'})
    data_recovered_pivot.reset_index(inplace=True)
    
    # Merge to single dataframe
    all_data = pd.merge(data_confirmed_pivot, data_deaths_pivot, 'left', on = ["index"] )
    all_data = pd.merge(all_data, data_recovered_pivot, 'left', on = ["index"] )
    all_data.reset_index(inplace=True)
    
    population = int(df_countries.loc[df_countries['Country_Region'] == country]['Population'].sum())

    # count rest of population not affected by virus (population - confirmed)
    # as confirmed = currently sick + deaths + recovered
    all_data['brak'] = 0
    all_data['aktywne'] = 0
    for i in range(len(all_data)):
        all_data.loc[i,'brak'] = population - all_data.loc[i,'potwierdzone']
        all_data.loc[i,'aktywne'] = all_data.loc[i,'potwierdzone'] - all_data.loc[i,'śmiertelne'] - all_data.loc[i,'wyzdrowienia']
    
    # Draw chart
    fig = px.bar(
            all_data.loc[all_data['index'] == data], 
            y='index',
            x=['aktywne', 'śmiertelne','wyzdrowienia','brak'], 
            orientation='h',
            height=400,
            title="Całkiwita liczba przypadków covid-19 na przekroju populacji kraju w "+country, 
            labels={
                 "index": "Dzień",
                 "value": "Całkowita liczba przypadków",
                 "variable": "Rodzaj przypadku"
            },
            color_discrete_sequence=['#636efa', '#ef553b', '#00cc96', '#fafafa'])
    fig.show()

interactive(children=(DatePicker(value=datetime.date(2021, 1, 7), description='Dzień:'), Dropdown(description=…

Dla Polski możemy zaobserwować, że na tle całej populacji liczba zachorowań na dzień sporządzenia raportu liczba chorych wynosi ok 230 tysięcy, a osób które wyzdrowiały ponad 1,01 miliona. Do tej pory liczba zgonów, gdzie przyczyną był covid-19 wynosi ok 30 tysięcy.

### Liczba przypadków w danym okresie czasu w wybranym kraju

Wygenerowany wykres pudełkowy przedstawia podstawowe informacje o cechach statystycznych dotyczących przypadków zachorowań na covid-19 na podstawie wybranego fragmentu czasu oraz wybranego kraju.

In [425]:
@interact
def show_box_country_dates(
    country=widgets.Dropdown(
        options=sorted(df_countries['Country_Region'].unique()), 
        value=_("Poland"),
        description='Kraj:'
    ),
    typ=widgets.Dropdown(
        options=['potwierdzone', 'śmiertelne', 'wyzdrowienia'], 
        value='potwierdzone',
        description='Rodzaj przypadku:'
    ),
    date_from=widgets.DatePicker(
        disabled=False,
        description='Data od:',
        value=datetime.date(2020, 9, 1),
    ),
    date_to=widgets.DatePicker(
        disabled=False,
        description='Data do:',
        value=datetime.date(2020, 12, 31),
    )):
    
    # Convert data from datePicker to correct format
    date_from_s = date_from.strftime('%Y/%m/%d')
    date_to_s = date_to.strftime('%Y/%m/%d')
    
    # Validate date
    if (date_from_s not in df_confirmed.columns):
        return '"Data od" spoza zakresu: '+df_confirmed.columns[2] + " - " + df_confirmed.columns[-1]
    if (date_to_s not in df_confirmed.columns):
        return '"Data do" spoza zakresu: '+df_confirmed.columns[2] + " - " + df_confirmed.columns[-1]
    if (date_to < date_from):
        return '"Data od" powinna być mniejsza niż "Data do".'

    # Choose correct dataframe
    if typ == 'potwierdzone':  
        df = df_confirmed
    elif typ == 'śmiertelne':
        df = df_deaths
    else:
        df = df_recovered
    
    # Choose correct dataframe
    if typ == 'potwierdzone':  
        df = df_confirmed
        color = ['#636efa']
    elif typ == 'śmiertelne':
        df = df_deaths
        color = ['#ef553b']
    else:
        df = df_recovered
        color = ['#00cc96']
        
    # Process dataframe
    df = pd.pivot_table(df.loc[df['Country/Region'] == country], 
                          columns=['Country/Region'], 
                          aggfunc=sum, 
                          fill_value=0).rename(columns={country: 'total'})
    df.reset_index(inplace=True)
    
    # Set start and end date
    start = df.loc[df['index'] == date_from_s].index.tolist()[0]
    end = df.loc[df['index'] == date_to_s].index.tolist()[0]
    
    # Count daily cases for selected type
    last = None
    df[typ] = 0
    for i in range(len(df)):
        if (i == 0):
            continue
        df.loc[i,typ] = df.loc[i,'total'] - df.loc[i-1,'total']
    
    # Select only date range
    df = df.loc[start:end]
    fig = go.Figure()
    fig.add_trace(go.Box(y=df[typ], name=typ,
                marker_color = color[0]))
    fig.update_layout(title_text='Wykres pudełkowy\ndzienna liczba zachorowań '+typ+' w kraju '+country
                      +"\nw przedziale dni od "+date_from_s+" do "+date_to_s)
    fig.show()

    

interactive(children=(Dropdown(description='Kraj:', index=135, options=('Afganistan', 'Albania', 'Algieria', '…

W okresie od początku września do końca grudnia 2020 roku dzienna liczba zaraportowanych nowych przypadków zachorowań na covid-19 w Polsce wynosiła od 302 do 32 tysiące. Mediana zachorowań dla tego okresu wynosi około 9tysięcy, a przez 25% dni omaianego  okresu dzienna liczba nowych zachorowań wynosiła ponad 15tysięcy.